- Show sums of inputs/outputs for only one Anlage
- Show how many entries do have percentage but no total numbers for all inputs/outputs
- Pro Gerätekategorie: 
    - Wie viel geht verloren zwischen den Stationen
    - Oder kommt noch was dazu
    - Wie geht die Bilanz auf
    - Nur für Kunststoff

In [257]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.sankey as sankey
import plotly.graph_objects as go

In [258]:
path1 = 'database1.csv'
path2 = 'database2.csv'
path3 = 'database3.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)

# Replace NaN in cells with emptry string in all dataframes
# df1.fillna('', inplace=True)
# df2.fillna('', inplace=True)
# df3.fillna('', inplace=True)

In [259]:
# To show all columns
pd.set_option('display.max_columns', None)
df1 = df1.fillna('')

# Eine weitere Spalte erstellen (ANLAGE), die den Namen des Betriebs + dessen Ort enthält
df1["ANLAGE"]=df1["BETRIEB_NAME"] + " " + df1["BETRIEB_ORT"]


# Move new Column (ANLAGE) to the start of the dataframe
df1.insert(0, 'ANLAGE', df1.pop('ANLAGE'))


print("DataFrame from database1.csv:\n")
print(df1.head(10))

DataFrame from database1.csv:

                                              ANLAGE  \
0                          technocycle GmbH Bramsche   
1                Lars Walch GmbH & Co. KG Baudenbach   
2     Lars Walch GmbH & Co. GmbH & Co. KG Baudenbach   
3                Lars Walch GmbH & Co. KG Baudenbach   
4                Lars Walch GmbH & Co. KG Baudenbach   
5          FSD Lwerk Berlin Brandenburg gGmbH Berlin   
6  HKV Elektroschrott Recycling GmbH & Co. KG Mai...   
7                    Hausgeräte Lagerverkauf Dorsten   
8            BRAL Reststoff-Bearbeitungs GmbH Berlin   
9   Weinand Recyclingges. mbH & Co. KG Bad Kreuznach   

                                 BETRIEB_NAME      BETRIEB_STRASSE  \
0                            technocycle GmbH     Industriestrasse   
1                    Lars Walch GmbH & Co. KG     Raiffeisenstraße   
2         Lars Walch GmbH & Co. GmbH & Co. KG    Raiffeisenstraße    
3                    Lars Walch GmbH & Co. KG     Raiffeisenstraße   
4 

C:\Users\Felix Medicus\AppData\Local\Temp\ipykernel_21360\1058597048.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1["ANLAGE"]=df1["BETRIEB_NAME"] + " " + df1["BETRIEB_ORT"]
C:\Users\Felix Medicus\AppData\Local\Temp\ipykernel_21360\1058597048.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1.insert(0, 'ANLAGE', df1.pop('ANLAGE'))


## Bilanzen für Gerätekategorie x

In [260]:
def sum_float_values(column):
    total = 0.0
    for value in column:
        if isinstance(value, (float, int)):
            total += value
    return total

In [261]:
# Funktion um zu schauen ob Totaler Eintrag oder Prozent Eintrag leer ist

def determine_missing_vals(dataframe, total_value_col, perc_value_col):

    missing_values = {"total_missing": 0, "perc_missing": 0}
    
    # Check if the specified columns exist in the DataFrame
    if total_value_col not in dataframe.columns or perc_value_col not in dataframe.columns:
        raise ValueError("One or both of the specified columns do not exist in the DataFrame.")

    # Iterate through the rows and perform the comparison
    for index, row in dataframe.iterrows():
        if row[total_value_col] == '' and row[perc_value_col] != '':
            missing_values["total_missing"] += 1
        elif row[total_value_col] != '' and row[perc_value_col] == '':
            missing_values["perc_missing"] += 1

    return missing_values

### Nach einzelnen Anlagen Filtern

In [262]:
# Nach einzelnen Anlagen (Name Betrieb + Ort des Betriebs) schauen. Falls alle Anlagen betrachtet werden sollen, das Feld leer lassen.
# Es kann auch nach Betrieben gesucht werden (dann werden alle Anlagen des Betrieb mit einbezogen)
anlage = ""

if len(anlage) != 0:
    df1 = df1[df1['ANLAGE'].str.lower().str.contains(anlage.lower())]
    
print("Anzahl der Observationen für die gegebene Anlange: ", len(df1))

Anzahl der Observationen für die gegebene Anlange:  363


### Gesamter Input für Gerätekategorie x in EBA

In [263]:
category = "KG4"
inputCategory = category + "_INPUT"
# Gesamter Input 
KG_Input_Sum = sum_float_values(df1[inputCategory])
print(KG_Input_Sum)

221816.0


### Output der EBA für Gerätekategorie x

In [264]:
column = category + "_OUT_GES_MASSE"
ges_out_eba = sum_float_values(df1[column])
print("Gesamter Output der EBA für Kategorie", category, ": ", ges_out_eba)

column = category + "_OUT_KS_MASSE"
ks_out_eba = sum_float_values(df1[column])
print("Davon Kunststoffe", category, ": ", ks_out_eba)

Gesamter Output der EBA für Kategorie KG4 :  150642.0
Davon Kunststoffe KG4 :  24536.0


## EBA die Output in Prozent aber nicht Tonnen haben

In [265]:
df_filtered = df1[df1[inputCategory] != '']

determine_missing_vals(df_filtered, "KG1_OUT_KS_MASSE", "KG1_OUT_KS_MASSE_KONTR")

{'total_missing': 0, 'perc_missing': 0}

### Input in VWA für Gerätekategorie x

In [266]:
column = category + "_TRT_GES"
ges_in_vwa = sum_float_values(df1[column])
print("Gesamter Input für VWA für Kategorie", category, ": ", ges_in_vwa)

column = category + "_TRT_KS"
ks_in_vwa = sum_float_values(df1[column])
print("Kunststoff Input für VWA für Kategorie", category, ": ", ks_in_vwa)

column = category + "_TRT_RCY"
rec_in_vwa = sum_float_values(df1[column])
print("Recycling Input in VWA für Kategorie", category, ": ", rec_in_vwa)

column = category + "_TRT_RCY_KS"
rec_ks_in_vwa = sum_float_values(df1[column])
print("Recycling Kunststoff Input in VWA für Kategorie", category, ": ", rec_ks_in_vwa)

column = category + "_TRT_RCV_OTH"
sonVW_in_vwa = sum_float_values(df1[column])
print("Sonstige Verwertung Input in VWA für Kategorie", category, ": ", sonVW_in_vwa)

column = category + "_TRT_RCV_OTH_KS"
sonVW_ks_in_vwa = sum_float_values(df1[column])
print("Sonstige Verwertung Kunststoff Input in VWA für Kategorie", category, ": ", sonVW_ks_in_vwa)



Gesamter Input für VWA für Kategorie KG4 :  141521.0
Kunststoff Input für VWA für Kategorie KG4 :  20392.0
Recycling Input in VWA für Kategorie KG4 :  114112.0
Recycling Kunststoff Input in VWA für Kategorie KG4 :  8145.0
Sonstige Verwertung Input in VWA für Kategorie KG4 :  44043.0
Sonstige Verwertung Kunststoff Input in VWA für Kategorie KG4 :  12342.0


### Anzahl an Inputwerten in VWA wo entweder Totaler Wert oder prozentualer Wert nicht angegeben ist

In [267]:
df_filtered = df1[df1[inputCategory] != '']

determine_missing_vals(df_filtered, "KG1_TRT_KS", "KG1_TRT_KONTR")

{'total_missing': 0, 'perc_missing': 0}

### Outputs der VWA für Gerätekategorie x

In [268]:
column = category + "_OUT_RCY_GES"
ges_rec_out_vwa = sum_float_values(df1[column])
print("Gesamter Recycling Output der VWA für Kategorie", category, ": ", ges_rec_out_vwa)
column = category + "_OUT_RCY_KS"
ks_rec_out_vwa = sum_float_values(df1[column])
print("Recycling Kunststoff Output der VWA für Kategorie", category, ": ", ks_rec_out_vwa)

Gesamter Recycling Output der VWA für Kategorie KG4 :  128215.0
Recycling Kunststoff Output der VWA für Kategorie KG4 :  10177.0


### Anzahl an Outputwerten aus VWA wo entweder Totaler Wert oder prozentualer Wert nicht angegeben ist

In [269]:
df_filtered = df1[df1[inputCategory] != '']

determine_missing_vals(df_filtered, "KG1_OUT_RCY_KS", "KG1_OUT_RCY_ANTEIL")


{'total_missing': 0, 'perc_missing': 0}

### Losses

In [270]:
# Gesamter Output der EBA die nicht in VWA eingeht
out_eba_in_vwa = ges_out_eba - ges_in_vwa
print("Output EBA der nicht in Input VWA ist: ", out_eba_in_vwa)

# Kunstoff Output der EBA die nicht in VWA eingeht
ks_out_eba_in_vwa = ks_out_eba - ks_in_vwa
print("Kunstoff output EBA der nicht in Input VWA ist: ", ks_out_eba_in_vwa)

Output EBA der nicht in Input VWA ist:  9121.0
Kunstoff output EBA der nicht in Input VWA ist:  4144.0


In [271]:
# Anteil des Recycling Input am gesamten Input in VWA
an_ges_rec_in_vwa = rec_in_vwa/ges_in_vwa
print("Anteil des Recycling Inputs am Gesamtinput in die VWA: ", an_ges_rec_in_vwa)

# Anteil des Recycling Inputs am gesamten Input in VWA für Kunststoffe
an_ks_rec_in_vwa = rec_ks_in_vwa/ks_in_vwa
print("Anteil des Recycling Inputs am Gesamtinput in die VWA für Kunststoffe: ", an_ks_rec_in_vwa)

Anteil des Recycling Inputs am Gesamtinput in die VWA:  0.8063255629906516
Anteil des Recycling Inputs am Gesamtinput in die VWA für Kunststoffe:  0.3994213417026285


In [272]:
# Gesamter Recycling Input in VWA vs. gesamter Recycling Output aus VWA
ges_rec_in_vwa_rec_out_vwa = rec_in_vwa - ges_rec_out_vwa
print("Recycling Input in VWA der nicht als recycelter Output auftritt: ", ges_rec_in_vwa_rec_out_vwa)

# Gesamter Recyling Input in VWA vs. gesamter Recycling Output aus VWA für Kunststoffe
ks_rec_in_vwa_rec_out_vwa = rec_ks_in_vwa - ks_rec_out_vwa
print("Recycling Input in VWA der nicht als recycelter Output auftritt für Kunststoffe: ", ks_rec_in_vwa_rec_out_vwa)

Recycling Input in VWA der nicht als recycelter Output auftritt:  -14103.0
Recycling Input in VWA der nicht als recycelter Output auftritt für Kunststoffe:  -2032.0
